In [2]:
import cv2
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from mahotas.features import surf
import os

In [4]:
cat_images = []
dog_images = []
horse_images = []

def load_images_from_folder(folder):
    img_features = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder, filename),cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img,(120,120))
        img_features.append(img)
    return img_features

cat_images = load_images_from_folder("D:\data\cats")
dog_images = load_images_from_folder("D:\data\dogs")
horse_images = load_images_from_folder("D:\data\horses")

In [5]:
def bagofwords(descriptors):
    X = np.vstack(descriptors)
    scaler = StandardScaler()
    X = scaler.fit_transform(X)
    kmeans = KMeans(n_clusters=100, random_state=42).fit(X)
    # Initialize a bag-of-words dictionary
    bow = np.zeros((len(descriptors), 100))

    for i, features in enumerate(descriptors):
        labels = kmeans.predict(features)
        bow[i] = np.bincount(labels, minlength=100)
    return bow

In [6]:
def get_sift_features(image):
    sift = cv2.SIFT_create()
    keypoints, descriptors = sift.detectAndCompute(image, None)
    if descriptors is not None:
            return descriptors
            descriptors = np.mean(descriptors,axis=0)
            return descriptors

def get_surf_features(image):
    descriptors = surf.surf(image)
    if (descriptors is not None):
        return descriptors
        descriptors = np.mean(descriptors,axis=0)
        return descriptors

def get_hog_features(img):
    cell_size = (16, 16)  # h x w in pixels
    block_size = (2, 2)  # h x w in cells
    nbins = 9  # number of orientation bins
    hog = cv2.HOGDescriptor(_winSize=(img.shape[1] // cell_size[1] * cell_size[1],
                                      img.shape[0] // cell_size[0] * cell_size[0]),
                            _blockSize=(block_size[1] * cell_size[1],
                                        block_size[0] * cell_size[0]),
                            _blockStride=(cell_size[1], cell_size[0]),
                            _cellSize=(cell_size[1], cell_size[0]),
                            _nbins=nbins)
    hog_feat = hog.compute(img)
    return hog_feat.flatten()

In [7]:
cat_features_sift = np.array([get_sift_features(img) for img in cat_images])
dog_features_sift = np.array([get_sift_features(img) for img in dog_images])
horse_features_sift = np.array([get_sift_features(img) for img in horse_images])
cat_features_sift = bagofwords(cat_features_sift)
dog_features_sift = bagofwords(dog_features_sift)
horse_features_sift = bagofwords(horse_features_sift)


cat_features_surf = np.array([get_surf_features(img) for img in cat_images])
dog_features_surf = np.array([get_surf_features(img) for img in dog_images])
horse_features_surf = np.array([get_surf_features(img) for img in horse_images])
cat_features_surf = bagofwords(cat_features_surf)
dog_features_surf = bagofwords(dog_features_surf)
horse_features_surf = bagofwords(horse_features_surf)

cat_features_hog = np.array([get_hog_features(img) for img in cat_images])
dog_features_hog = np.array([get_hog_features(img) for img in dog_images])
horse_features_hog = np.array([get_hog_features(img) for img in horse_images])


C:\Users\Ali Syed\AppData\Local\Temp\ipykernel_18076\2651460793.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  cat_features_sift = np.array([get_sift_features(img) for img in cat_images])
C:\Users\Ali Syed\AppData\Local\Temp\ipykernel_18076\2651460793.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  dog_features_sift = np.array([get_sift_features(img) for img in dog_images])
C:\Users\Ali Syed\AppData\Local\Temp\ipykernel_18076\2651460793.py:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tupl

In [8]:
X_sift = np.concatenate([cat_features_sift, dog_features_sift, horse_features_sift])
X_surf = np.concatenate([cat_features_surf, dog_features_surf, horse_features_surf])
X_hog = np.concatenate([cat_features_hog, dog_features_hog, horse_features_hog])
y = np.concatenate([np.zeros((len(cat_features_sift),)),np.ones((len(dog_features_sift),))*2,np.ones((len(horse_features_sift),))*2])

In [1]:
X_train_sift, X_test_sift, y_train_sift, y_test_sift = train_test_split(X_sift, y, test_size=0.3, random_state=42)
X_train_surf, X_test_surf, y_train_surf, y_test_surf = train_test_split(X_surf, y, test_size=0.3, random_state=42)
X_train_hog, X_test_hog, y_train_hog, y_test_hog = train_test_split(X_hog, y, test_size=0.3, random_state=42)

NameError: name 'train_test_split' is not defined

In [9]:
models = {
    'ANN':MLPClassifier(hidden_layer_sizes=(100,), max_iter=2000, alpha=0.0001, solver='adam', random_state=42),
    'RF':RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)
}
#'surf':[X_train_surf, X_test_surf, y_train_surf, y_test_surf]
category = {'sift': [X_train_sift,X_test_sift,y_train_sift,y_test_sift],'surf':[X_train_surf, X_test_surf, y_train_surf, y_test_surf], 'hog':[X_train_hog,X_test_hog,y_train_hog,y_test_hog]}

In [10]:
for i in models:
    for k in category:
        a = models[i]
        a.fit(category[k][0], category[k][2])
        y_pred_mlp = a.predict(category[k][1])
        accuracy_mlp = accuracy_score(category[k][3], y_pred_mlp)
        print(f'{i:4}+ {k:5}:', accuracy_mlp)

ANN + sift : 0.9590163934426229
ANN + surf : 1.0
ANN + hog  : 0.7704918032786885
RF  + sift : 0.8278688524590164
RF  + surf : 1.0
RF  + hog  : 0.7131147540983607
